Given a connection to a blank Postgres database, these scripts will create the necessary tables and then copy the data in the .csv files to the database.

Note that uploading data to a remote AWS Postgres instance is a little tricky - see [here](https://stackoverflow.com/questions/46969474/using-python-to-upload-large-csv-files-to-postgres-rds-in-aws/46969475#46969475) for more info

In [ ]:
## Add folder one level up to system path so Python can 'see' the db_connections module
import sys
sys.path.append('..')

from db_connections.connections import get_conn, runsql 
from db_connections.connections import host, dbname, username

In [ ]:
with open("sql/create_abp_tables.sql") as f:
    sql = " ".join(f.readlines())

# THE FOLLOWING STATEMENT DROPS ALL AddressBase Premium TABLES in the database, so only use it if you really want to start from scratch
# i.e. the following line needs to be run the first time you run this script, but be careful not to accidentally run it again, because it will delete everything in the database!
# runsql(sql)  

In [ ]:
# Now need to read each table in 
copy_list = [('abp_blpu', 'ID21_BLPU_Records.csv'),
('abp_delivery_point', 'ID28_DPA_Records.csv'),
('abp_lpi', 'ID24_LPI_Records.csv'),
('abp_crossref', 'ID23_XREF_Records.csv'),
('abp_classification', 'ID32_Class_Records.csv'),
('abp_street', 'ID11_Street_Records.csv'),
('abp_street_descriptor', 'ID15_StreetDesc_Records.csv'),
('abp_organisation', 'ID31_Org_Records.csv'),
('abp_successor', 'ID30_Successor_Records.csv')]

The following might take about 40 minutes on a very fast connection.

Note that I tried to use multithreading to make it faster, but it didn't increase the upload speed.  The maximum I was about to achive on the MoJ Digital Wifi was about 15MB/s upload.

In [ ]:
import os
import datetime
import subprocess

for row in copy_list:
    command = "\copy {} FROM 'raw/{}' DELIMITER ',' CSV HEADER".format(row[0], row[1])
    psql_template = 'psql -p 5432 --host {} --username {} --dbname {} --command "{}"'
    bash_command = psql_template.format(host, username, dbname, command.strip())
    
    print("---")
    print(datetime.datetime.now().time())
    print(bash_command)
    print("Creating table {} from {}".format(row[0], row[1]))

    process = subprocess.Popen(bash_command, stdout=subprocess.PIPE, shell=True) 
 
    output, error = process.communicate()
    print(output)
    print(error)
    print("---")



Check that the number of records in the remote tables equals the number of lines in the csv files

In [ ]:
import pandas as pd 

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


sql = """
select count(*) from {}
"""
df_rows = []

conn = get_conn()
for row in copy_list:

    table_name =  row[0]
    df = pd.read_sql(sql.format(table_name), conn)
    count = df.iloc[0,0]
    line_count = file_len("raw/"+row[1]) - 1 #-1 to account for header
    df_rows.append({"table_name": table_name, 
                    "count": count,
                    "line_count": line_count})
    
pd.DataFrame(df_rows)
